In [90]:
import pandas as pd 

In [91]:
dataset=pd.read_excel("data/data_test.xlsx")

In [92]:
dataset=dataset.drop("Unnamed: 0",axis=1)

In [94]:
dataset=dataset.drop_duplicates()

In [95]:
dataset.head(2)

,Photo produit 1,Photo produit 2,Nom produit,Lien achat,Catégorie produit,Genre,cielab_colors
0,https://wasted.fr/cdn/shop/files/t-shirt-acid-...,-,T-Shirt Acid,https://wasted.fr/collections/t-shirts-homme/p...,T-shirt,H/F,"[[10.588235294117647, 0.0, 1.0]]"
1,https://wasted.fr/cdn/shop/files/t-shirt-happy...,-,T-Shirt Happy When it Rains,https://wasted.fr/collections/t-shirts-homme/p...,T-shirt,H/F,"[[10.588235294117647, 0.0, 0.0]]"


In [96]:
dataset.columns

Index(['Photo produit 1', 'Photo produit 2 ', 'Nom produit ', 'Lien achat',
       'Catégorie produit', 'Genre', 'cielab_colors'],
      dtype='object')

In [98]:
import ast 

dataset["cielab_colors"]=dataset["cielab_colors"].apply(ast.literal_eval)

In [99]:
data=dataset.explode("cielab_colors").reset_index(drop=True)

In [100]:
data.shape


(1782, 7)

In [101]:
import faiss 

In [102]:
import numpy as np 

In [152]:
all_colors=np.array(data["cielab_colors"].tolist(),dtype="float32")

In [104]:
dim=3

In [159]:
import faiss
import numpy as np



def normalize_L2(vectors):
    """Convertit en float32 et applique la normalisation L2"""
    vectors = np.array(vectors, dtype=np.float32)  # Assurer float32
    faiss.normalize_L2(vectors)  # Normalisation
    return vectors


# Charger les couleurs CIELAB (déjà normalisées entre 0 et 1)


# Normalisation pour la similarité cosinus
dataset_cielab_normalized = normalize_L2(all_colors)

# Créer un index FAISS avec similarité cosinus
d = 3 # Dimension des couleurs
index = faiss.IndexFlatIP(d)  # Index pour la similarité cosinus
index.add(dataset_cielab_normalized)  # Ajouter les vecteurs normalisés



In [160]:
# Sauvegarde de l'index Faiss
faiss.write_index(index, "faiss_index.idx")

In [44]:
def fin_matching_products(query_colors,index,df,top_K=5):
    query_colors=np.array(query_colors,dtype="float32")
    distances,indices=index.search(query_colors,top_K)
    matches=[]
    for i,query in enumerate(query_colors):
        for j in range(top_K):
            product_id=df.iloc[indices[i][j]["Photo produit 1"]]
            distance=distances[i][j]
            matches.append((product_id,distance))

    product_distances={}
    for product_id,distance in matches:
        if product_id not in product_distances or distance < product_distances[product_id]:
            product_distances[product_id]=distance
    sorted_products=sorted(product_distances.items(),key=lambda x: x[1])
    return sorted_products[:top_K]

In [139]:
print(dataset[dataset["cielab_colors"]=="[[58.03921568627452, -5.0, 26.0], [46.27450980392157, -3.0, 23.0]]"])

Empty DataFrame
Columns: [Photo produit 1, Photo produit 2 , Nom produit , Lien achat, Catégorie produit, Genre, cielab_colors]
Index: []


In [125]:
query_colors=[[58.03921568627452, -5.0, 26.0], [46.27450980392157, -3.0, 23.0]]

In [108]:
# Sauvegarde des ID pour retrouver les produits plus tard
product_ids = np.array(data["Photo produit 1"])

In [149]:
def filter_results_by_threshold(D, I, threshold=90):
    """
    Convertit les distances en score et filtre les résultats en dessous du seuil.
    """
    results = []
    for i, distances in enumerate(D):
        for j, dist in enumerate(distances):
            score = max(0, 100 - dist)  # Convertir la distance en score
            if score >= threshold:
                print(score)
                results.append((I[i][j], score))
    return results


In [150]:
from collections import Counter

def match_similar_products(query_colors, k=5, threshold=90):
    """
    - Détecte et normalise les couleurs de l'image
    - Filtre les catégories compatibles
    - Effectue le matching avec FAISS et filtre les scores < 90%
    """
    # Étape 1 : Détection et normalisation
    query_colors = np.array([normalize_color(c) for c in query_colors])  
    


    # Étape 3 : Recherche des couleurs dans Faiss
    product_matches = []
    for query_color in query_colors:
        D, I = index.search(np.array([query_color]), k=5)  # Recherche Faiss

        # Filtrer les résultats sous le seuil de 90%
        filtered_results = filter_results_by_threshold(D, I, threshold)
        matched_products = [product_ids[idx] for idx, score in filtered_results ]
        
        product_matches.extend(matched_products)  # Ajout à la liste globale
    
    # Étape 4 : Agrégation des résultats
    product_counts = Counter(product_matches)
    sorted_products = sorted(product_counts.items(), key=lambda x: x[1], reverse=True)

    return sorted_products[:k]



In [168]:
print(dataset.iloc[150,:][6])

[[63.921568627450974, -5.0, -17.0]]


C:\Users\BahaARFAOUI\AppData\Local\Temp\ipykernel_15116\954741250.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(dataset.iloc[150,:][6])


In [156]:
query_color = [[63.921568627450974, -5.0, -17.0]]

In [151]:
match_similar_products(query_colors=query_colors)

100.0
99.999245
99.99849
99.997986
99.997986
100.0
99.99937
99.9982
99.99802
99.997986


[('https://nestorevintage.fr/cdn/shop/files/307FDA5D-D3FB-48D6-8DA2-2478DF104BD3.jpg?v=1738348432&width=2048',
  2),
 ('https://nestorevintage.fr/cdn/shop/files/412D63EB-6B35-495D-85BA-F26344D3024F.jpg?v=1737575887&width=2048',
  2),
 ('https://nestorevintage.fr/cdn/shop/files/2A6F83FB-EC8C-47F1-9B44-017383A513C6.jpg?v=1737897164&width=2048',
  2),
 ('https://nestorevintage.fr/cdn/shop/files/91D912F0-3E3C-40DA-AC96-20D2A1533068.jpg?v=1737896991&width=2048',
  1),
 ('https://fr.sandro-paris.com/dw/image/v2/BCMW_PRD/on/demandware.static/-/Sites-master-catalog/default/dwb6adda94/images/packshot/Sandro_SFPPA01544-84_F_P.jpg?sw=650&sh=650',
  1)]

In [161]:
def match_similar_products_cosine(query_colors, k=5, threshold=0.9):
    """
    - Détecte et normalise les couleurs de l'image
    - Filtre les catégories compatibles
    - Effectue le matching avec FAISS (cosine similarity)
    - Garde uniquement les résultats avec un score ≥ 90%
    """
    # Étape 1 : Détection et normalisation des couleurs
    query_colors = np.array([normalize_color(c) for c in query_colors])  

  

    # Étape 3 : Recherche des couleurs dans Faiss
    product_matches = []
    for query_color in query_colors:
        query_color = normalize_L2(np.array([query_color]))  # Normalisation L2 avant recherche
        D, I = index.search(query_color, k=20)  # Recherche FAISS
        
        # Convertir les distances en scores de similarité
        scores = D[0]  # Les scores sont déjà entre -1 et 1
        filtered_results = [(I[0][i], scores[i]) for i in range(len(scores)) if scores[i] >= threshold]

        matched_products = [product_ids[idx] for idx, score in filtered_results]
        product_matches.extend(matched_products)

    # Étape 4 : Agrégation des résultats
    product_counts = Counter(product_matches)
    sorted_products = sorted(product_counts.items(), key=lambda x: x[1], reverse=True)

    return sorted_products[:k]


In [163]:
matched_products = match_similar_products_cosine(query_color, k=5, threshold=0.9)

print("Produits recommandés :\n\n\n", matched_products)

Produits recommandés :


 [('https://nestorevintage.fr/cdn/shop/files/6F63EA32-B320-4B16-93C0-ED31F37DE5E4.jpg?v=1730051143&width=2048', 1), ('https://wasted.fr/cdn/shop/files/pantalon-casper-acid-corduroy-952014.jpg?v=1721324562', 1), ('https://nestorevintage.fr/cdn/shop/files/C3A18182-357E-4551-8383-A9E57DD20141.jpg?v=1720365983&width=2048', 1), ('https://fr.sandro-paris.com/dw/image/v2/BCMW_PRD/on/demandware.static/-/Sites-master-catalog/default/dw06039427/images/packshot/Sandro_SFPOU00697-G023_F_P.jpg?sw=650&sh=650', 1), ('https://nestorevintage.fr/cdn/shop/files/IMG_05622_4b3d4fad-7021-4cc8-991a-fc508babefa2.jpg?v=1722790441&width=2048', 1)]


In [164]:
data[data["Photo produit 1"]=="https://nestorevintage.fr/cdn/shop/files/6F63EA32-B320-4B16-93C0-ED31F37DE5E4.jpg?v=1730051143&width=2048"]

,Photo produit 1,Photo produit 2,Nom produit,Lien achat,Catégorie produit,Genre,cielab_colors
1711,https://nestorevintage.fr/cdn/shop/files/6F63E...,https://nestorevintage.fr/cdn/shop/files/E36A0...,Portefeuille MiuMiu,https://nestorevintage.fr/products/portefeuill...,Portefeuille,F,"[47.84313725490196, 18.0, 16.0]"
1712,https://nestorevintage.fr/cdn/shop/files/6F63E...,https://nestorevintage.fr/cdn/shop/files/E36A0...,Portefeuille MiuMiu,https://nestorevintage.fr/products/portefeuill...,Portefeuille,F,"[30.58823529411765, 19.0, 20.0]"


In [165]:
import faiss
import numpy as np

# --------------------------------------
# 1. Prepare Data for FAISS
# --------------------------------------
# Extract all color vectors as a numpy array
all_colors = np.array(data["cielab_colors"].tolist(), dtype="float32")

# Create a FAISS index (L2 distance for CIELAB)
dim = 3 # L, a, b
index = faiss.IndexFlatL2(dim)
index.add(all_colors)


In [170]:
# --------------------------------------
# 2. Query Function
# --------------------------------------
def find_matching_products(query_colors, index, df, top_k=5):
    query_colors = np.array(query_colors, dtype="float32")
    distances, indices = index.search(query_colors, top_k)

    # Collect all matching product IDs and their distances
    matches = []
    for i, query in enumerate(query_colors):
        for j in range(top_k):
            product_id = df.iloc[indices[i][j]]["Photo produit 1"]
            distance = distances[i][j]
            matches.append((product_id, distance))

    # Group by Product ID and keep the best (min) distance
    product_distances = {}
    for product_id, distance in matches:
        if product_id not in product_distances or distance < product_distances[product_id]:
            product_distances[product_id] = distance

    # Sort products by distance
    sorted_products = sorted(product_distances.items(), key=lambda x: x[1])
    return sorted_products[:top_k]


In [171]:
# --------------------------------------
# 3. Example Usage
# --------------------------------------
# Input: Assume 2 dominant colors
query_colors = query_color = [[63.921568627450974, -5.0, -17.0]] # Example CIELAB vectors
matches = find_matching_products(query_colors, index, data, top_k=5)


In [172]:
matches

[('https://wasted.fr/cdn/shop/files/varsity-jacket-prey-742723.jpg?v=1737390429',
  np.float32(0.0)),
 ('https://ntmb.it/cdn/shop/products/jeans-baggy-reworked-blue-311475.jpg?v=1694471080&width=3000',
  np.float32(7.8446774)),
 ('https://nestorevintage.fr/cdn/shop/files/055917A6-267F-48CA-87FE-973C1946E906.jpg?v=1725801192&width=2048',
  np.float32(14.384086)),
 ('https://fr.sandro-paris.com/dw/image/v2/BCMW_PRD/on/demandware.static/-/Sites-master-catalog/default/dwcd63739a/images/packshot/Sandro_SHPBE00082-40_H_P.jpg?sw=650&sh=650',
  np.float32(16.153788)),
 ('https://nestorevintage.fr/cdn/shop/files/IMG_23132.jpg?v=1715529282&width=2048',
  np.float32(18.384087))]

In [173]:
from sklearn.neighbors import NearestNeighbors

In [174]:
nn=NearestNeighbors(n_neighbors=5,metric="euclidean")

In [175]:
nn.fit(all_colors)

NearestNeighbors(metric='euclidean')

In [179]:
def find_matching_products_nn(query_colors, nn_model, df, top_k=5):
    query_colors = np.array(query_colors, dtype="float32")
    distances, indices = nn_model.kneighbors(query_colors)

    # Collect all matching product IDs and their distances
    matches = []
    for i, query in enumerate(query_colors):
        for j in range(top_k):
            product_id = df.iloc[indices[i][j]]["Photo produit 1"]
            distance = distances[i][j]
            matches.append((product_id, distance))

    # Group by Product ID and keep the best (min) distance
    product_distances = {}
    for product_id, distance in matches:
        if product_id not in product_distances or distance < product_distances[product_id]:
            product_distances[product_id] = distance

    # Sort products by distance
    sorted_products = sorted(product_distances.items(), key=lambda x: x[1])
    return sorted_products[:top_k]


In [185]:
query_colors = query_color = [[83.13725490196079, 0.0, 0.0], [70.58823529411765, 0.0, 0.0]] # Example CIELAB vectors
matches = find_matching_products(query_colors, index, data, top_k=5)

In [186]:
matches

[('https://wasted.fr/cdn/shop/files/crewneck-negative-617699_2048x2048.jpg?v=1712661665',
  np.float32(0.0)),
 ('https://wasted.fr/cdn/shop/products/jean-casper-snow-feeler-762622.jpg?v=1706147807',
  np.float32(0.0)),
 ('https://wasted.fr/cdn/shop/files/t-shirt-united-930779_2048x2048.jpg?v=1730548101',
  np.float32(0.1537883)),
 ('https://wasted.fr/cdn/shop/products/hoodie-zip-crown-pitcher-736902_2048x2048.jpg?v=1689958848',
  np.float32(0.1537883)),
 ('https://wasted.fr/cdn/shop/files/jean-casper-signature-610785.jpg?v=1737390239',
  np.float32(0.1537883))]

In [187]:
dataset.iloc[140,][0]

C:\Users\BahaARFAOUI\AppData\Local\Temp\ipykernel_15116\271524657.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dataset.iloc[140,][0]


'https://wasted.fr/cdn/shop/products/jean-casper-snow-feeler-762622.jpg?v=1706147807'